In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, random_split
from conversion import CSVToTensor

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input_size = 9
        self.output_size = 9
        self.hidden_size = 2*27

        self.W1 = nn.Parameter(torch.randn(self.input_size, self.hidden_size))
        self.b1 = nn.Parameter(torch.randn(self.hidden_size))
        self.W2 = nn.Parameter(torch.randn(self.hidden_size, self.output_size))
        self.b2 = nn.Parameter(torch.randn(self.output_size))
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.3)
        self.crossloss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.train_data = None
        self.val_data = None
    
    def forward(self, x):
        x = x @ self.W1 + self.b1
        x = self.relu(x)
        x = x @ self.W2 + self.b2
        # x = self.dropout(x)
        return x

    def load_data(self, file_path):
        dataloader = CSVToTensor(file_path)
        dataloader.create_all_tensor()
        dataset = dataloader.create_a_dataset()

        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        self.train_data, self.val_data = random_split(dataset, [train_size, val_size])
        self.train_data = DataLoader(self.train_data, batch_size=16, shuffle=True)
        self.val_data = DataLoader(self.val_data, batch_size=16, shuffle=True)

    def train_model(self, epochs):
        self.to(self.device)
        self.train()
        for epoch in range(epochs):
            epoch_loss = 0
            for src, trg in self.train_data:
                src = src.to(self.device)
                trg = trg.to(self.device)
                self.optimizer.zero_grad()
                output = self.forward(src)
                loss = self.crossloss(output, trg.argmax(dim=1))
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(self.train_data)
            print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}")
    
    

    

In [21]:
if __name__ == '__main__':
    model = Model()
    # choose the dataset file path
    model.load_data('./Datasets/tic_tac_toe_500_games.csv')
    # choose the number of epochs
    with torch.no_grad():
        model.W1.copy_(torch.zeros(model.input_size, model.hidden_size))
        model.b1.copy_(torch.ones(model.hidden_size))
        model.W2.copy_(torch.zeros(model.hidden_size, model.output_size))
        model.b2.copy_(torch.ones(model.output_size))
    model.train_model(11)

Epoch: 0, Loss: 2.2083
Epoch: 1, Loss: 2.2043
Epoch: 2, Loss: 2.1989
Epoch: 3, Loss: 2.1995
Epoch: 4, Loss: 2.1927
Epoch: 5, Loss: 2.1876
Epoch: 6, Loss: 2.1841
Epoch: 7, Loss: 2.1793
Epoch: 8, Loss: 2.1736
Epoch: 9, Loss: 2.1712
Epoch: 10, Loss: 2.1681
